# Program for Recommendation System 

### Install Surprise Library

In [1]:
# !pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-macosx_10_9_universal2.whl size=1585307 sha256=b3a366437055ea9f6a22fc6c6872cceaa1d650e39e6beb71b96e409e60fd2b47
  Stored in directory: /Users/godspeed/Library/Caches/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


### Implementation

In [19]:
from surprise import Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the MovieLens dataset
data = Dataset.load_builtin('ml-100k')

# Define the algorithm (Singular Value Decomposition - SVD)
algo = SVD()

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# Train the algorithm on the training set
algo.fit(trainset)

# Make predictions on the test set
predictions = algo.test(testset)

# Evaluate the accuracy of the model
accuracy.rmse(predictions)

# Get recommendations for a specific user (user ID: 196)
user_id = str(196)
user_ratings = [(trainset.to_raw_iid(item), algo.predict(user_id, item).est) for item in trainset.all_items() if item not in trainset.ur[trainset.to_inner_uid(user_id)]]
sorted_ratings = sorted(user_ratings, key=lambda x: x[1], reverse=True)

print(f"Top 5 recommended movies for user {user_id}:")
for movie_id, predicted_rating in sorted_ratings[:5]:
    print(f"Movie ID: {movie_id}, Predicted Rating: {predicted_rating:.2f}")


RMSE: 0.9430
Top 5 recommended movies for user 196:
Movie ID: 773, Predicted Rating: 3.47
Movie ID: 1028, Predicted Rating: 3.47
Movie ID: 1078, Predicted Rating: 3.47
Movie ID: 421, Predicted Rating: 3.47
Movie ID: 288, Predicted Rating: 3.47


### Implementation of Collaborative Filtering

In [18]:
import numpy as np

class CollaborativeFiltering:
    def __init__(self, num_users, num_items):
        self.num_users = num_users
        self.num_items = num_items
        self.user_item_matrix = np.zeros((num_users, num_items))
        self.similarity_matrix = np.zeros((num_users, num_users))

    def add_interaction(self, user_id, item_id, rating):
        self.user_item_matrix[user_id, item_id] = rating

    def calculate_similarity(self):
        for i in range(self.num_users):
            for j in range(self.num_users):
                if i != j:
                    # Calculate cosine similarity between users i and j
                    numerator = np.dot(self.user_item_matrix[i], self.user_item_matrix[j])
                    denominator = np.linalg.norm(self.user_item_matrix[i]) * np.linalg.norm(self.user_item_matrix[j])
                    self.similarity_matrix[i, j] = numerator / (denominator + 1e-10)

    def recommend_items(self, user_id, num_recommendations=5):
        # Find the most similar users to the target user
        similar_users = np.argsort(self.similarity_matrix[user_id])[::-1]

        # Identify items that the target user hasn't interacted with
        unrated_items = np.where(self.user_item_matrix[user_id] == 0)[0]

        # Recommend items based on the most similar users
        recommendations = []
        for item_id in unrated_items[:num_recommendations]:
            predicted_rating = np.mean(self.user_item_matrix[similar_users[:3], item_id])
            recommendations.append((item_id, round(predicted_rating,2)))

        # Sort recommendations by predicted rating
        recommendations.sort(key=lambda x: x[1], reverse=True)

        return recommendations

# Example usage
num_users = 4
num_items = 5

# Create a CollaborativeFiltering instance
cf = CollaborativeFiltering(num_users, num_items)

# Add user-item interactions (user_id, item_id, rating)
interactions = [
    (0, 0, 5), (0, 1, 4), (0, 3, 2),
    (1, 0, 3), (1, 2, 5), (1, 3, 1),
    (2, 1, 2), (2, 3, 4),
    (3, 2, 3), (3, 4, 5)
]

for user, item, rating in interactions:
    cf.add_interaction(user, item, rating)

# Calculate user similarity
cf.calculate_similarity()

# Recommend items for a specific user
user_id_to_recommend = 2
recommendations = cf.recommend_items(user_id_to_recommend, num_recommendations=3)

print(f"Recommendations for User {user_id_to_recommend}: {recommendations}")


Recommendations for User 2: [(0, 2.67), (2, 2.67), (4, 1.67)]
